In [1]:
!git clone https://github.com/datitran/raccoon_dataset.git

Cloning into 'raccoon_dataset'...
remote: Enumerating objects: 652, done.
remote: Total 652 (delta 0), reused 0 (delta 0), pack-reused 652
Receiving objects: 100% (652/652), 48.01 MiB | 28.30 MiB/s, done.
Resolving deltas: 100% (415/415), done.


In [2]:
!git clone https://github.com/tensorflow/models.git

Cloning into 'models'...
remote: Enumerating objects: 67051, done.
remote: Total 67051 (delta 0), reused 0 (delta 0), pack-reused 67051
Receiving objects: 100% (67051/67051), 576.29 MiB | 32.07 MiB/s, done.
Resolving deltas: 100% (47069/47069), done.


In [3]:
%cd /content/models/research/
!protoc object_detection/protos/*.proto --python_out=.
# Install TensorFlow Object Detection API.
!cp object_detection/packages/tf2/setup.py .
!python -m pip install .

/content/models/research
Processing /content/models/research
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
     |████████████████████████████████| 9.9 MB 9.8 MB/s 
     |████████████████████████████████| 352 kB 55.0 MB/s 
     |████████████████████████████████| 1.8 MB 59.0 MB/s 
     |████████████████████████████████| 23.1 MB 1.3 MB/s 
     |████████████████████████████████| 99 kB 10.6 MB/s 
     |████████████████████████████████| 4.9 MB 49.8 MB/s 
     |████████████████████████████████| 596 kB 60.5 MB/s 
     |████████████████████████████████| 213 kB 61.3 MB/s 
     |████████████████████████████████| 1.1 MB 51.8 MB/s 
     |█████████████████

In [23]:
import tensorflow as tf
import keras
import os
import pandas as pd
import numpy as np
from collections import namedtuple
import io
from PIL import Image
import sys
sys.path.append(os.path.abspath("./object_detection"))
from keras.preprocessing import image
from object_detection.utils import dataset_util

In [12]:
def class_text_to_int(row_label):
    if row_label == 'raccoon':
        return 1
    else:
        None

def split(df, group):
    data = namedtuple('data', ['filename', 'object'])
    gb = df.groupby(group)
    return [data(filename, gb.get_group(x)) for filename, x in zip(gb.groups.keys(), gb.groups)]


def create_tf_example(group, path):
    with tf.io.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
        encoded_jpg = fid.read()
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = Image.open(encoded_jpg_io)
    width, height = image.size

    filename = group.filename.encode('utf8')
    image_format = b'jpg'
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    for index, row in group.object.iterrows():
        xmins.append(row['xmin'] / width)
        xmaxs.append(row['xmax'] / width)
        ymins.append(row['ymin'] / height)
        ymaxs.append(row['ymax'] / height)
        classes_text.append(row['class'].encode('utf8'))
        classes.append(class_text_to_int(row['class']))

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example


def generate_tfrecord(output_path, image_dir, csv_input):
    writer = tf.io.TFRecordWriter(output_path)
    path = os.path.join(image_dir)
    examples = pd.read_csv(csv_input)
    grouped = split(examples, 'filename')
    for group in grouped:
        tf_example = create_tf_example(group, path)
        writer.write(tf_example.SerializeToString())

    writer.close()
    output_path = os.path.join(os.getcwd(), output_path)
    print('Successfully created the TFRecords: {}'.format(output_path))

In [37]:
!mkdir my_checkpoints
!wget https://storage.googleapis.com/mobilenet_v2/checkpoints/mobilenet_v2_1.4_224.tgz 
!mv mobilenet_v2_1.4_224.tgz my_checkpoints
!cd my_checkpoints
!tar -xvf mobilenet_v2_1.4_224.tgz
!rm mobilenet_v2_1.4_224.tgz
!cd ..

--2022-01-15 23:31:07--  https://storage.googleapis.com/mobilenet_v2/checkpoints/mobilenet_v2_1.4_224.tgz
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.141.128, 2607:f8b0:4023:c0b::80
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.141.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 135977516 (130M) [application/x-tar]
Saving to: ‘mobilenet_v2_1.4_224.tgz’

mobilenet_v2_1.4_22 100%[===================>] 129.68M  62.8MB/s    in 2.1s    

2022-01-15 23:31:10 (62.8 MB/s) - ‘mobilenet_v2_1.4_224.tgz’ saved [135977516/135977516]

tar: mobilenet_v2_1.4_224.tgz: Cannot open: No such file or directory
tar: Error is not recoverable: exiting now
rm: cannot remove 'mobilenet_v2_1.4_224.tgz': No such file or directory


In [24]:
generate_tfrecord('/content/train.record', '/content/raccoon_dataset/images' , '/content/raccoon_dataset/data/train_labels.csv')


Successfully created the TFRecords: /content/train.record


In [25]:
!wget http://download.tensorflow.org/models/object_detection/classification/tf2/20200710/mobilenet_v2.tar.gz
!tar -xvf mobilenet_v2.tar.gz

--2022-01-15 22:46:44--  http://download.tensorflow.org/models/object_detection/classification/tf2/20200710/mobilenet_v2.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 142.250.141.128, 2607:f8b0:4023:c0b::80
Connecting to download.tensorflow.org (download.tensorflow.org)|142.250.141.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8404070 (8.0M) [application/x-tar]
Saving to: ‘mobilenet_v2.tar.gz’

mobilenet_v2.tar.gz 100%[===================>]   8.01M  --.-KB/s    in 0.1s    

2022-01-15 22:46:45 (53.6 MB/s) - ‘mobilenet_v2.tar.gz’ saved [8404070/8404070]

mobilenet_v2/
mobilenet_v2/mobilenet_v2.ckpt-1.index
mobilenet_v2/checkpoint
mobilenet_v2/mobilenet_v2.ckpt-1.data-00001-of-00002
mobilenet_v2/mobilenet_v2.ckpt-1.data-00000-of-00002


In [32]:
from __future__ import print_function
from IPython import display 
checkpoint_name = 'mobilenet_v2_1.0_224' #@param
url = 'https://storage.googleapis.com/mobilenet_v2/checkpoints/' + checkpoint_name + '.tgz'
print('Downloading from ', url)
!wget {url}
print('Unpacking')
!tar -xvf {checkpoint_name}.tgz
checkpoint = checkpoint_name + '.ckpt'

display.clear_output()
print('Successfully downloaded checkpoint from ', url,
      '. It is available as', checkpoint)

Successfully downloaded checkpoint from  https://storage.googleapis.com/mobilenet_v2/checkpoints/mobilenet_v2_1.0_224.tgz . It is available as mobilenet_v2_1.0_224.ckpt


In [72]:
%%bash
PIPELINE_CONFIG_PATH='/content/config.config'
MODEL_DIR='/content/model/'
NUM_TRAIN_STEPS=1000
SAMPLE_1_OF_N_EVAL_EXAMPLES=1
python /content/models/research/object_detection/model_main_tf2.py \
    --pipeline_config_path=${PIPELINE_CONFIG_PATH} \
    --model_dir=${MODEL_DIR} \
    --num_train_steps=${NUM_TRAIN_STEPS}

2022-01-15 23:51:42.985983: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
W0115 23:51:42.987592 140143025870720 cross_device_ops.py:1387] There are non-GPU devices in `tf.distribute.Strategy`, not using nccl allreduce.
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
I0115 23:51:42.988761 140143025870720 mirrored_strategy.py:376] Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
INFO:tensorflow:Maybe overwriting train_steps: 1000
I0115 23:51:42.993989 140143025870720 config_util.py:552] Maybe overwriting train_steps: 1000
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I0115 23:51:42.994180 140143025870720 config_util.py:552] Maybe overwriting use_bfloat16: False
Instructions for updating:
rename to distribute_datasets_from_function
W0115 23:51:43.037226 140143025870720 deprecation.py:347] From /usr/local/

In [27]:
pip install opencv-python-headless==4.1.2.30

     |████████████████████████████████| 21.8 MB 1.3 MB/s 
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.5.5.62
    Uninstalling opencv-python-headless-4.5.5.62:
      Successfully uninstalled opencv-python-headless-4.5.5.62


In [73]:
!zip -r content.zip content/ 


	zip warning: name not matched: content/

zip error: Nothing to do! (try: zip -r content.zip . -i content/)


In [74]:
%cd ..


/content/models/research


In [75]:
%cd ..

/content/models


In [76]:
%cd ..


/content


In [77]:
%cd ..

/


In [78]:
!zip -r content.zip content/ 

  adding: content/ (stored 0%)
  adding: content/.config/ (stored 0%)
  adding: content/.config/active_config (stored 0%)
  adding: content/.config/config_sentinel (stored 0%)
  adding: content/.config/.last_update_check.json (deflated 24%)
  adding: content/.config/.last_opt_in_prompt.yaml (stored 0%)
  adding: content/.config/gce (stored 0%)
  adding: content/.config/configurations/ (stored 0%)
  adding: content/.config/configurations/config_default (deflated 15%)
  adding: content/.config/.last_survey_prompt.yaml (stored 0%)
  adding: content/.config/logs/ (stored 0%)
  adding: content/.config/logs/2022.01.07/ (stored 0%)
  adding: content/.config/logs/2022.01.07/14.32.37.114755.log (deflated 91%)
  adding: content/.config/logs/2022.01.07/14.33.22.232212.log (deflated 54%)
  adding: content/.config/logs/2022.01.07/14.32.57.118850.log (deflated 53%)
  adding: content/.config/logs/2022.01.07/14.33.15.504888.log (deflated 86%)
  adding: content/.config/logs/2022.01.07/14.33.42.565580.l

mobilenet_v2_1.4_224.ckpt.data-00000-of-00001  mobilenet_v2_1.4_224_frozen.pb
mobilenet_v2_1.4_224.ckpt.index                mobilenet_v2_1.4_224_info.txt
mobilenet_v2_1.4_224.ckpt.meta                 mobilenet_v2_1.4_224.tflite
mobilenet_v2_1.4_224_eval.pbtxt                mobilenet_v2_1.4_224.tgz
